In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd

In [2]:
archivo = 'GH70_501_filtered_by_seq_kmers_2_spaced_1.csv'
df_original = pd.read_csv(archivo,sep=',',)
df_modificado = df_original.set_index("name")
df_transpose = df_modificado.transpose()

In [3]:
columnas = df_transpose.columns
for i in range(len(columnas)):
    viejo = columnas[i]
    nuevo = viejo.replace('.','_')
    dic = {viejo:nuevo}
    df_transpose = df_transpose.rename(columns=dic)
df_transpose = df_transpose.reset_index()

In [4]:
spark = SparkSession\
        .builder\
        .appName("Kmeans")\
        .getOrCreate()

22/11/10 17:58:02 WARN Utils: Your hostname, darian-Laptop resolves to a loopback address: 127.0.1.1; using 10.0.85.1 instead (on interface outline-tun0)
22/11/10 17:58:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/10 17:58:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/10 17:58:03 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 97:76:b4:1c:c6:ae:3c:6c


In [5]:
df_spark = spark.createDataFrame(df_transpose,)

In [6]:
col = df_spark.columns[1:]

In [7]:
assambler = VectorAssembler(inputCols=col,outputCol='features')
df_Features = assambler.transform(df_spark)

In [8]:
#Buscar el mejor entre los 10 primeros
#k_mejor = 0
#silueta_anterior = 0
#for i in range(2,11):
#    kmeans = KMeans(featuresCol='features',k=i)
#    model = kmeans.fit(df_Features)
#    pred = model.transform(df_Features)
#    eval = ClusteringEvaluator()
#    silhouette = eval.evaluate(pred)
#    print(f"{i}-->{silhouette}")
#    if silhouette > silueta_anterior:
#        k_mejor = i
#        silueta_anterior = silhouette

22/11/10 17:58:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


2-->0.6573294150703497
3-->0.4545948013639467
4-->0.4433098899304613
5-->0.4472595279126809
6-->0.41463241106833393
7-->0.26854406376056256
8-->0.29006717696438084
9-->0.2977854109679544
10-->0.2611124718001811


In [9]:
kmeans = KMeans(featuresCol='features',k=2)
model = kmeans.fit(df_Features)

In [10]:
pred = model.transform(df_Features)

In [11]:
eval = ClusteringEvaluator()
silhouette = eval.evaluate(pred)
print(f"Silueta: {silhouette}")

Silueta: 0.6573294150703497


In [12]:
pred.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  274|
|         0|  126|
+----------+-----+



In [13]:
df_cero = pred[pred['prediction']==0]
df_cero = df_cero.drop(*('features','prediction'))
df_uno = pred[pred['prediction']==1]
df_uno = df_uno.drop(*('features','prediction'))

In [14]:
df_final0 = df_cero.toPandas()
df_final1 = df_uno.toPandas()

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

...

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/home/darian/anaconda3/envs/bigdata/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

In [15]:
columnas = df_final0.columns
for i in range(len(columnas)):
    viejo = columnas[i]
    nuevo = viejo.replace('_','.')
    dic = {viejo:nuevo}
    df_final0 = df_final0.rename(columns=dic)
dic = {"index":"name"}
df_final0 = df_final0.rename(columns=dic)

In [16]:
columnas = df_final1.columns
for i in range(len(columnas)):
    viejo = columnas[i]
    nuevo = viejo.replace('_','.')
    dic = {viejo:nuevo}
    df_final1 = df_final1.rename(columns=dic)
dic = {"index":"name"}
df_final1 = df_final1.rename(columns=dic)

In [17]:
df_nuevo = df_final0.set_index("name")
df_Tfinal0 = df_nuevo.transpose()

df_nuevo = df_final1.set_index("name")
df_Tfinal1 = df_nuevo.transpose()

In [18]:
df_Tfinal0.to_csv('kmers_2_spaced_1--Grupo-0-Final.csv',header=True,index=True)
df_Tfinal1.to_csv('kmers_2_spaced_1--Grupo-1-Final.csv',header=True,index=True)

In [19]:
spark.stop()